![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/regression-part2-automated-ml.png)

## Download and prepare data

In [4]:
import pandas as pd
taxi_ds = pd.DataFrame([])

In [5]:
from azureml.opendatasets import NycTlcGreen
from datetime import datetime
from dateutil.relativedelta import relativedelta
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_ds = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    taxi_ds = taxi_ds.append(temp_ds.sample(2000))
    
taxi_ds.head(10)

ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=1/part-00175-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2745-1.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=164655.82 [ms]
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=2/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=2/part-00007-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2577-1.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=124977.06 [ms]
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=3/']
Looking for parquet files...
Reading them into Pandas dataframe...
Re

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
1096327,2,2015-01-08 09:27:48,2015-01-08 09:53:48,2,3.92,None,None,-73.956230,40.733658,-73.997940,...,1,19.0,0.0,0.5,0.3,3.0,5.33,NaN,28.13,1.0
114615,2,2015-01-12 09:22:23,2015-01-12 09:41:46,1,3.25,None,None,-73.927376,40.758152,-73.972122,...,2,14.5,0.0,0.5,0.3,0.0,0.00,NaN,15.30,1.0
879953,1,2015-01-05 17:08:02,2015-01-05 17:17:52,1,2.00,None,None,-73.878876,40.879555,-73.880211,...,2,9.0,1.0,0.5,0.3,0.0,0.00,NaN,10.80,1.0
410163,2,2015-01-19 17:46:38,2015-01-19 17:49:42,1,0.67,None,None,-73.948502,40.803520,-73.941658,...,2,4.5,0.0,0.5,0.3,0.0,0.00,NaN,5.30,1.0
307080,1,2015-01-03 19:15:08,2015-01-03 19:19:03,1,0.70,None,None,-73.978012,40.684296,-73.980995,...,2,5.0,0.0,0.5,0.3,0.0,0.00,NaN,5.80,1.0
636285,2,2015-01-04 04:14:14,2015-01-04 04:24:44,1,3.36,None,None,-73.975647,40.693398,-74.005035,...,1,12.0,0.5,0.5,0.3,0.0,0.00,NaN,13.30,1.0
1209945,2,2015-01-25 22:31:50,2015-01-25 22:46:52,1,3.23,None,None,-73.949974,40.675831,-73.993233,...,1,13.0,0.5,0.5,0.3,2.7,0.00,NaN,17.00,1.0
419998,2,2015-01-19 22:59:31,2015-01-19 23:22:01,1,10.20,None,None,-73.910423,40.757534,-73.790909,...,2,30.0,0.5,0.5,0.3,0.0,0.00,NaN,31.30,1.0
559253,1,2015-01-22 01:14:15,2015-01-22 01:22:00,1,1.60,None,None,-73.957611,40.671478,-73.938881,...,2,8.0,0.5,0.5,0.3,0.0,0.00,NaN,9.30,1.0
1075537,2,2015-01-03 03:32:26,2015-01-03 03:34:36,1,0.43,None,None,-73.936325,40.804016,-73.942291,...,2,3.5,0.5,0.5,0.3,0.0,0.00,NaN,4.80,1.0


In [6]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

taxi_ds[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = taxi_ds[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
taxi_ds.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
1096327,2,2015-01-08 09:27:48,2015-01-08 09:53:48,2,3.92,None,None,-73.956230,40.733658,-73.997940,...,0.3,3.0,5.33,NaN,28.13,1.0,1,8,3,9
114615,2,2015-01-12 09:22:23,2015-01-12 09:41:46,1,3.25,None,None,-73.927376,40.758152,-73.972122,...,0.3,0.0,0.00,NaN,15.30,1.0,1,12,0,9
879953,1,2015-01-05 17:08:02,2015-01-05 17:17:52,1,2.00,None,None,-73.878876,40.879555,-73.880211,...,0.3,0.0,0.00,NaN,10.80,1.0,1,5,0,17
410163,2,2015-01-19 17:46:38,2015-01-19 17:49:42,1,0.67,None,None,-73.948502,40.803520,-73.941658,...,0.3,0.0,0.00,NaN,5.30,1.0,1,19,0,17
307080,1,2015-01-03 19:15:08,2015-01-03 19:19:03,1,0.70,None,None,-73.978012,40.684296,-73.980995,...,0.3,0.0,0.00,NaN,5.80,1.0,1,3,5,19
636285,2,2015-01-04 04:14:14,2015-01-04 04:24:44,1,3.36,None,None,-73.975647,40.693398,-74.005035,...,0.3,0.0,0.00,NaN,13.30,1.0,1,4,6,4
1209945,2,2015-01-25 22:31:50,2015-01-25 22:46:52,1,3.23,None,None,-73.949974,40.675831,-73.993233,...,0.3,2.7,0.00,NaN,17.00,1.0,1,25,6,22
419998,2,2015-01-19 22:59:31,2015-01-19 23:22:01,1,10.20,None,None,-73.910423,40.757534,-73.790909,...,0.3,0.0,0.00,NaN,31.30,1.0,1,19,0,22
559253,1,2015-01-22 01:14:15,2015-01-22 01:22:00,1,1.60,None,None,-73.957611,40.671478,-73.938881,...,0.3,0.0,0.00,NaN,9.30,1.0,1,22,3,1
1075537,2,2015-01-03 03:32:26,2015-01-03 03:34:36,1,0.43,None,None,-73.936325,40.804016,-73.942291,...,0.3,0.0,0.00,NaN,4.80,1.0,1,3,5,3


Remove some of the columns that you won't need for training or additional feature building.

In [7]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    taxi_ds.pop(col)
    
taxi_ds.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
1096327,2,2,3.92,-73.956230,40.733658,-73.997940,40.756611,28.13,1,8,3,9
114615,2,1,3.25,-73.927376,40.758152,-73.972122,40.753845,15.30,1,12,0,9
879953,1,1,2.00,-73.878876,40.879555,-73.880211,40.856525,10.80,1,5,0,17
410163,2,1,0.67,-73.948502,40.803520,-73.941658,40.812939,5.30,1,19,0,17
307080,1,1,0.70,-73.978012,40.684296,-73.980995,40.675766,5.80,1,3,5,19


### Clean the data 

In [9]:
taxi_ds.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.781333,1.378917,2.888657,-73.839630,40.696608,-73.838266,40.694571,14.796296,6.500000,15.163625,3.265458,13.561292
std,0.413351,1.052601,2.958490,2.655885,1.464741,2.655985,1.464706,11.562651,3.452124,8.484354,1.963263,6.785978
min,1.000000,0.000000,0.000000,-74.187897,0.000000,-74.192726,0.000000,-9.300000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.060000,-73.959496,40.698935,-73.967203,40.699363,7.880000,3.750000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.900000,-73.945045,40.746861,-73.944210,40.747654,11.300000,6.500000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.620000,-73.916195,40.803174,-73.908987,40.792246,17.800000,9.250000,22.000000,5.000000,19.000000
max,2.000000,6.000000,85.100000,0.000000,41.201145,0.000000,41.316383,364.840000,12.000000,30.000000,6.000000,23.000000


In [10]:
final_df = taxi_ds.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

In [11]:
final_df.describe()

,vendorID,passengerCount,tripDistance,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,23297.000000,23297.000000,23297.000000,23297.000000,23297.000000,23297.000000,23297.000000,23297.000000
mean,1.782247,1.381637,2.945026,14.798411,6.506460,15.154741,3.268490,13.561703
std,0.412728,1.057252,2.873239,10.320914,3.453262,8.488531,1.964437,6.792606
min,1.000000,1.000000,0.250000,0.100000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.100000,8.160000,4.000000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.950000,11.300000,7.000000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.700000,17.800000,10.000000,22.000000,5.000000,19.000000
max,2.000000,6.000000,27.310000,150.650000,12.000000,30.000000,6.000000,23.000000


## Configure workspace


In [12]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


## Split the data into train and test sets

In [13]:
from sklearn.model_selection import train_test_split

y_df = final_df.pop("totalAmount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

## Automatically train a model

### Define training settings

In [14]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.ERROR,
    "n_cross_validations": 5
}

In [15]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             X=x_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

WARNING - The AutoMLConfig inputs you have specified will soon be deprecated. Please use the AutoMLConfig shown in our documentation: https://aka.ms/AutoMLConfig


### Train the automatic regression model

In [16]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_969d2ec4-2644-484a-b054-b9c7a37412dd
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing values imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.

**************************************************************

## Explore the results

In [17]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Retrieve the best model

In [18]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: taxi-experiment,
Id: AutoML_969d2ec4-2644-484a-b054-b9c7a37412dd_18,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('pref...6666667, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.2, 0.06666666666666667]))]),
          stddev=None)


### Test the best model accuracy

In [19]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

[ 9.0011987  24.13537629 27.88067837 13.55502016 13.45567149  9.7869719
  7.07544132 14.02201018 10.74296056 26.36530762]


In [20]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

3.5880866031145633

In [21]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.1397074922835672

Model Accuracy:
0.8602925077164328
